# Import needed libraries
- `pandas`
- `geopandas`

In [ ]:
import geopandas as gpd
import pandas as pd

# Import the SA2 and *map outlines*
- there are various ways of doing this
  - we can go to the [website](https://datafinder.stats.govt.nz/layer/106728-statistical-area-2-2022-generalised/) and press the download button
  ![image.png](Data2024Assets/DownloadSA2Website.jpg) 
  - we can use command-line tools directly from here, such as  `wget` or `curl` to download the data 
  - here, we will download the data `wget` from a GitHub repository that hosts an optimised derivative of the original data set

In [ ]:
!wget "https://github.com/UoA-eResearch/SA2_2022_population/raw/main/statistical-area-2-2023-generalised_simplified_22.3%25.zip"

## Let's have a look at the dataset we just downloaded

In [ ]:
sa2 = gpd.read_file("statistical-area-2-2023-generalised_simplified_22.3%.zip").dropna(subset="geometry")
sa2

We do not want to include the Chatam Islands, as they are not part of the main landmass of New Zealand and only under 800 people live there.

![alt](https://upload.wikimedia.org/wikipedia/commons/thumb/2/2c/Chatham_Islands_in_New_Zealand_%28zoom%29.svg/520px-Chatham_Islands_in_New_Zealand_%28zoom%29.svg.png)

In [ ]:
sa2 = sa2[(sa2.SA22023__1 != "Chatham Islands") & (sa2.LAND_AREA_ > 0)].copy()
sa2

# Import the population Dataset

Similarly, we download the dataset

In [ ]:
!wget "https://raw.githubusercontent.com/UoA-eResearch/SA2_2022_population/main/population_by_SA2.csv"

In [ ]:
population = pd.read_csv("population_by_SA2.csv")
population

The following is one example of how we can clean our two dataframes
- the outline (`sa2` dataframe) has the column `SA22023_V1` with one example value of `100100`
- our population file has the column `Area` with an example value of `100100 North Cape`
How to merge these?
We could try several manual approaches or counting (the first 6 digits, etc)

But:
- we use something called [Regex](https://en.wikipedia.org/wiki/Regular_expression) or Regular Expression
- keeping the scope of this session in mind, think of it as a defined set of rules that we can apply to a string to find/match things


  ``` 
  r'(\d+)': This is a raw string containing a regular expression.
  \d: Matches any digit (0-9).
  +: Matches one or more of the preceding element (in this case, one or more digits).
  (): Capturing group, indicating that we want to extract the digits.
  ```
- by using this, we can extract digits (any number of them greater 0) from a string
- and we put them together in one `capture group`

In [ ]:
# Extract ID from Area col and adding this to a new column on the right called 'SA2'
population['SA2'] = population['Area'].str.extract(r'(\d+)')
population

In [ ]:
# Add a prefix to the right dataframe's columns (excluding the merge key)
prefix = 'population_in_year_'
population= population.rename(columns={col: prefix + col for col in population.columns[1:10]})
population

# Ready, steady: Merge... Wait...

- OK, almost there
- Let's wait again
- How to merge the two datasets and how to make sure that the right data is in the right place?
- In other words: How can we explicitly state that we want to look for the column `SA22023_V1` in the `SA2` dataset and the column we named `SA2` in the `population]` dataset?
- By specifying it!

In [ ]:
sa2 = sa2.merge(population, left_on='SA22023_V1', right_on='SA2')
sa2

# Let's map

In [ ]:
m = sa2.explore("population_in_year_2022", legend=True)
m.save("index_folium.html")
m

It is likely that you will get an error such as:

*ImportError: The 'folium', 'matplotlib' and 'mapclassify' packages are required for 'explore()'*

We know what to do: `pip install` these

# Next steps:
- `pip freeze > requirements.txt` and inspect that file
- set GH repo up and publish